In [ ]:
import pandas as pd
import numpy as np
import cv2
from deepface import DeepFace
import mediapipe as mp
from pytube import YouTube

emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
excel_file = 'btp_list.xlsx'
sheet_name = 'Sheet1'
df = pd.read_excel(excel_file, sheet_name=sheet_name)
video_links = df['youtube_video_code']

# Check if 'resulted_emotions' column exists, if not, create it
if 'resulted_emotions' not in df.columns:
    df['resulted_emotions'] = ''
mp_drawing = mp.solutions.drawing_utils
# Initialize MediaPipe holistics
mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
# Iterate over each video
for video_url in video_links:
    # Check if 'resulted_emotions' column is non-empty for this video, if yes, skip it
    if not pd.isna(df.loc[df['youtube_video_code'] == video_url, 'resulted_emotions'].iloc[0]):
        print(f"Emotions already analyzed for video '{video_url}'. Skipping...")
        continue
    
    try:
        mp_pose = mp.solutions.pose
        pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

        
        # Load pre-trained emotion detection model
        # emotion_model = load_model('emotion_model.h5')  # Load your emotion detection model here

        # Create empty lists to store data for graphs
        shoulder_midpoints = []
        head_turn_angles = []
        left_hand = []
        right_hand = []
        
        yt = YouTube(f"wwww.youtube.com/watch?v={video_url}")
        video_stream = yt.streams.get_highest_resolution()
        # Open the video stream
        cap = cv2.VideoCapture(video_stream.url)

        # Get video properties
        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Initialize list to store resulted emotions
        resulted_emotions_list = []

        # Loop through each frame
        frame_count = 0
        
        while True:
            
            # Read the frame
            success, frame = cap.read()
            frame_count += 1
            if not success:
                break
            
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            if frame_count % int(fps) == 0:
                # print(video_url)
                # Analyze the frame for emotion
                # print('Analyzing frame..., frame_count:', frame_count)
                frame_result = DeepFace.analyze(img_path=frame_rgb, actions=['emotion'], detector_backend='opencv', enforce_detection=False)
                # Get the highest confident emotion
                if frame_result is not None:
                    highest_emotion = max(frame_result[0]['emotion'], key=frame_result[0]['emotion'].get)
                    resulted_emotions_list.append(highest_emotion)
                # else:
                    # resulted_emotions_list.append('none')
            # else:
                # resulted_emotions_list.append('none')
            results = pose.process(frame_rgb)

            if results.pose_landmarks is not None and frame_count % int(fps) == 0:
                # print('Analyzing pose..., frame_count:', frame_count)
                # Extract landmarks of the left and right shoulders (ids 11 and 12)
                left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
                right_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]

                if left_shoulder and right_shoulder:
                    # Calculate the midpoint of the shoulders
                    shoulder_midpoint_x = (left_shoulder.x + right_shoulder.x) / 2

                    # Append the shoulder midpoint to the list
                    shoulder_midpoints.append(str(shoulder_midpoint_x))
                # else:
                #     # Append None for frames with no shoulder points
                #     shoulder_midpoints.append('none')

                mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

                # Extract landmarks of the left and right eyes and nose
                left_eye = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EYE_INNER]
                right_eye = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EYE_INNER]
                nose = results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE]

                if left_eye and right_eye and nose:
                    # Calculate the angle between the line connecting the eyes and a horizontal reference line
                    eye_line_vector = np.array([right_eye.x - left_eye.x, right_eye.y - left_eye.y])
                    eye_left_nose_vector = np.array([nose.x-left_eye.x ,  nose.y-left_eye.y])
                    eye_right_nose_vector = np.array([right_eye.x - nose.x, right_eye.y - nose.y])

                    dot_product_left = np.dot(eye_line_vector, eye_left_nose_vector)
                    eye_line_magnitude = np.linalg.norm(eye_line_vector)
                    eye_left_nose_magnitude = np.linalg.norm(eye_left_nose_vector)

                    dot_product_right = np.dot(eye_line_vector, eye_right_nose_vector)
                    eye_right_nose_magnitude = np.linalg.norm(eye_right_nose_vector)

                    cosine_angle_left = dot_product_left / (eye_line_magnitude * eye_left_nose_magnitude)
                    cosine_angle_right = dot_product_right / (eye_line_magnitude * eye_right_nose_magnitude)

                    head_turn_angle = np.arccos(cosine_angle_right) * (180 / np.pi)

                    head_turn_angles.append(str(head_turn_angle))

            #     else:
            #         head_turn_angles.append('none')
            # else:
            #     shoulder_midpoints.append('none')
            #     head_turn_angles.append('none')

            # Update the frame index
            # frame_index += 1

            
            frame_rgb.flags.writeable = False
            hand_results = holistic_model.process(frame_rgb)
            frame_rgb.flags.writeable = True

            left_hand_sum_distance = 0
            if hand_results.left_hand_landmarks is not None and frame_count % int(fps) == 0:
                # print('Analyzing left hand..., frame_count:', frame_count)
                # Drawing Left hand Land Marks
                # mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

                mp_drawing.draw_landmarks( frame, hand_results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                left_hand_landmarks = hand_results.left_hand_landmarks.landmark
                for landmark in left_hand_landmarks:
                    x = landmark.x
                    y = landmark.y
                    z = landmark.z
                    distance = np.sqrt(x**2 + y**2 + z**2)
                    left_hand_sum_distance += distance
                    
                left_hand.append(str(left_hand_sum_distance))
            # else:
            #     left_hand.append('none')
            
            right_hand_sum_distance = 0
            if hand_results.right_hand_landmarks is not None and frame_count % int(fps) == 0:
                # print('Analyzing right hand..., frame_count:', frame_count)
                mp_drawing.draw_landmarks(frame, hand_results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS )
                right_hand_landmarks = hand_results.right_hand_landmarks.landmark
                for landmark in right_hand_landmarks:
                    x = landmark.x
                    y = landmark.y
                    z = landmark.z
                    distance = np.sqrt(x**2 + y**2 + z**2)
                    right_hand_sum_distance += distance
                right_hand.append(str(right_hand_sum_distance))
            # else:
            #     right_hand.append('none')
            
            # print(video_url)
        print('Emotions analyzed for video:', video_url)
            

        # Release the video capture object
        cap.release()
        # Add the resulted emotions list to the dataframe
        df.loc[df['youtube_video_code'] == video_url, 'resulted_emotions'] = ', '.join(resulted_emotions_list)
        df.loc[df['youtube_video_code'] == video_url, 'shoulder_midpoints'] = ', '.join(shoulder_midpoints)
        df.loc[df['youtube_video_code'] == video_url, 'head_turn_angles'] = ', '.join(head_turn_angles)
        df.loc[df['youtube_video_code'] == video_url, 'left_hand'] = ', '.join(left_hand)
        df.loc[df['youtube_video_code'] == video_url, 'right_hand'] = ', '.join(right_hand)

        # Save the updated dataframe to the Excel file
        df.to_excel(excel_file, index=False)

        

    except Exception as e:
        print(f"Error processing video '{video_url}': {e}. Skipping...")
        continue


In [ ]:
'''
I have a python list which has exactly 7 entries, but they repeat as time
series. I wish to create a matrix of 7x7 which indicates the frequency of
transition from emotion i to emotion j. Write a function which does this.
For example, if the input is:
emotions=['happy', 'sad', 'angry', 'sad', 'sad', 'happy', 'angry', 'happy',
          'sad', 'angry']

The sad row and happy column should be 1, because there is one transition.
Please note that sad to happy and happy to sad are different transitions.

'''

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
def transition_matrix_prob(emotions):
    # Create a dictionary to map emotions to integers
    emotion_map = {emotion: i for i, emotion in enumerate(emotion_labels)}
    # Create a list of integers from the input list of emotions
    emotions_int = [emotion_map[emotion] for emotion in emotions]
    # Create a matrix of zeros
    matrix = np.zeros((len(emotion_labels), len(emotion_labels)))
    # Loop through the list of integers and increment the matrix
    for i in range(len(emotions_int) - 1):
        matrix[emotions_int[i], emotions_int[i + 1]] += 1
    # # Calculate the total number of transitions for each emotion
    # total_transitions = matrix.sum(axis=1)
    # # Convert counts to probabilities
    # matrix_prob = matrix / total_transitions[:, np.newaxis]
    # Calculate the total number of transitions for each emotion
    total_transitions = matrix.sum()
    # Convert counts to probabilities
    matrix_prob = matrix / total_transitions
    # Create a DataFrame from the matrix
    df = pd.DataFrame(matrix_prob, columns=emotion_labels, index=emotion_labels)
    return df

# # Example usage
# emotions = ['happy', 'sad', 'angry', 'sad', 'sad', 'happy', 'angry', 'happy', 'sad', 'angry']
# print(transition_matrix_prob(emotions))
matrixof2dmatrices = []

file = 'btp_list.xlsx'
sheet_name = 'Sheet1'
# open
df = pd.read_excel(file, sheet_name=sheet_name)
# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Get the emotions from the 'resulted_emotions' column
    emotions = row['resulted_emotions'].split(', ')
    # Generate the transition matrix for the emotions
    matrix = transition_matrix_prob(emotions)
    # Display the matrix
    # print(f"Transition Matrix for video '{row['youtube_video_code']}': Speaker '{row['speaker_name']}' : category '{row['category']}'")
    # print(matrix)
    matrixof2dmatrices.append(matrix)


    # Generate the transition matrix
    transition_matrix = matrix

    # Plot the heatmap
    # plt.figure(figsize=(8, 6))
    # sns.heatmap(transition_matrix, annot=True, cmap='YlGnBu', fmt='.2f')
    # plt.title('Transition Matrix Heatmap')
    # plt.xlabel('To Emotion')
    # plt.ylabel('From Emotion')
    # plt.show()



In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Assuming you have a list of emotion transition matrices stored in 'data'
# Each matrix should be represented as a 2D NumPy array
# For example:
# data = [matrix1, matrix2, ..., matrixN]

# Convert list of matrices to a 2D array
data_array = np.array(matrixof2dmatrices)

# Reshape the matrices into 1D arrays
reshaped_data = data_array.reshape(data_array.shape[0], -1)

# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(reshaped_data)

# Define the number of clusters
n_clusters = 8

# Initialize K-means clustering algorithm
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
# Fit the model to the scaled data
kmeans.fit(scaled_data)

# Get cluster labels for each matrix
cluster_labels = kmeans.labels_

# Print cluster labels for each matrix
cluster_labels_list = []
for i, label in enumerate(cluster_labels):
    cluster_labels_list.append(label)

In [ ]:
file = 'btp_list.xlsx'
sheet_name = 'Sheet1'
# open
i = 0
df = pd.read_excel(file, sheet_name=sheet_name)
# Check if the number of rows in df matches the length of cluster_labels
if len(df) != len(cluster_labels_list):
    print("Error: Number of rows in DataFrame does not match the number of cluster labels.")
else:
    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        if i < len(cluster_labels_list):
            print(f"Transition Matrix for video '{row['youtube_video_code']}': Speaker '{row['speaker_name']}': belongs to cluster {cluster_labels_list[i]+1}")
            i += 1

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Assuming you have a list of emotion transition matrices stored in 'data'
# Each matrix should be represented as a 2D NumPy array
# For example:
# data = [matrix1, matrix2, ..., matrixN]

# Convert list of matrices to a 2D array
data_array = np.array(matrixof2dmatrices)

# Reshape the matrices into 1D arrays
reshaped_data = data_array.reshape(data_array.shape[0], -1)

# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(reshaped_data)

# Define the number of clusters
n_clusters = 8

# Initialize K-means clustering algorithm
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Fit the model to the scaled data
kmeans.fit(scaled_data)

# Get cluster labels for each matrix
cluster_labels = kmeans.labels_

# Perform PCA for visualization
pca = PCA(n_components=2)  # You can also try 3 components for 3D visualization
pca_result = pca.fit_transform(scaled_data)

# Plot clusters
plt.figure(figsize=(10, 7))
for i in range(n_clusters):
    plt.scatter(pca_result[cluster_labels == i, 0], pca_result[cluster_labels == i, 1], label=f'Cluster {i+1}')

plt.title('Clustering of Emotion Transition Matrices')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
'''
I have a python list which has exactly 7 entries, but they repeat as time
series. I wish to create a matrix of 7x7 which indicates the frequency of
transition from emotion i to emotion j. Write a function which does this.
For example, if the input is:
emotions=['happy', 'sad', 'angry', 'sad', 'sad', 'happy', 'angry', 'happy',
          'sad', 'angry']

The sad row and happy column should be 1, because there is one transition.
Please note that sad to happy and happy to sad are different transitions.

'''

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
def transition_matrix_prob(emotions):
    # Create a dictionary to map emotions to integers
    emotion_map = {emotion: i for i, emotion in enumerate(emotion_labels)}
    # Create a list of integers from the input list of emotions
    emotions_int = [emotion_map[emotion] for emotion in emotions]
    # Create a matrix of zeros
    matrix = np.zeros((len(emotion_labels), len(emotion_labels)))
    # Loop through the list of integers and increment the matrix
    for i in range(len(emotions_int) - 1):
        matrix[emotions_int[i], emotions_int[i + 1]] += 1
    # # Calculate the total number of transitions for each emotion
    # total_transitions = matrix.sum(axis=1)
    # # Convert counts to probabilities
    # matrix_prob = matrix / total_transitions[:, np.newaxis]
    # Calculate the total number of transitions for each emotion
    total_transitions = matrix.sum()
    # Convert counts to probabilities
    matrix_prob = matrix / total_transitions
    # Create a DataFrame from the matrix
    df = pd.DataFrame(matrix_prob, columns=emotion_labels, index=emotion_labels)
    return df

# # Example usage
# emotions = ['happy', 'sad', 'angry', 'sad', 'sad', 'happy', 'angry', 'happy', 'sad', 'angry']
# print(transition_matrix_prob(emotions))
matrixof2dmatrices = []

file = 'btp_list.xlsx'
sheet_name = 'Sheet1'
# open
df = pd.read_excel(file, sheet_name=sheet_name)
# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    if row['speaker_name'] == "Unbox Therapy" or row['speaker_name']=="TechWiser" or row['speaker_name']=="iJustine":
        # Get the emotions from the 'resulted_emotions' column
        emotions = row['resulted_emotions'].split(', ')
        # Generate the transition matrix for the emotions
        matrix = transition_matrix_prob(emotions)
        # Display the matrix
        print(f"Transition Matrix for video '{row['youtube_video_code']}': Speaker '{row['speaker_name']}' : category '{row['category']}'")
        # print(matrix)
        matrixof2dmatrices.append(matrix)


        # Generate the transition matrix
        transition_matrix = matrix

        # Plot the heatmap
        plt.figure(figsize=(8, 6))
        sns.heatmap(transition_matrix, annot=True, cmap='YlGnBu', fmt='.2f')
        plt.title('Transition Matrix Heatmap')
        plt.xlabel('To Emotion')
        plt.ylabel('From Emotion')
        plt.show()

